In [7]:
import numpy as np
import pandas as pd
import openpyxl

# Lese die Excel-Datei mit dem Profil, welches varriert werden soll ein


#df_profil = pd.read_excel("profil_spring.xlsx")
#df_profil = pd.read_excel("profil_winter.xlsx")
#df_profil = pd.read_excel("profil_summer.xlsx")
df_profil = pd.read_excel("profil_fall.xlsx")

# Erstelle ein leeres Pandas-DataFrame für die Ergebnisse
results = pd.DataFrame()

# Für jedes Szenario:   
for i in range(1000):
    # Erstelle ein Array mit normalverteilten Zufallsvariablen für jeden Bus und Jahreszeit
    random_variables = np.random.normal(0, 0.002, (len(df_profil), 1))
    
    # Berechne die Summe der Zufallsvariablen
    sum_random_variables = sum(random_variables)
       
    # Berechne den Faktor, um den jede Zufallsvariable angepasst werden muss, damit die Summe 0 ergibt
    factor = -sum_random_variables / len(random_variables)

    # Passe jede Zufallsvariable an, indem der Faktor multipliziert wird
    random_variables_adjusted = random_variables + factor

    # Die gewünschte Toleranz weil die Berechnung von Gleitkommazahlen ungenau ist und somit die Summe der angepassten Zufallszahlen minimal von 0 abweichen kann
    tolerance = 1e-10

    # Prüfe, ob die Summe der angepassten Zufallsvariablen innerhalb der Toleranz ist
    assert np.allclose(np.sum(random_variables_adjusted, axis=0), 0, atol=tolerance)
    
    # Füge die Zufallsvariablen den profilparametern hinzu
    df_varied = df_profil.iloc[:, 1:] + random_variables_adjusted

    # Setze die Spaltenüberschriften auf "Scenario i"
    df_varied.columns = ["S{}".format(i+1)]
    
    # Füge die veränderten Lastparameter dem DataFrame der Ergebnisse hinzu
    results = pd.concat([results, df_varied], axis=1)
#    results_T = results.T    
      
# Speichere das DataFrame der Ergebnisse in einer Excel-Datei
#results.to_excel("Ergebnisse_2.xlsx", index=False)
#results_T.to_excel("Ergebnisse.xlsx", index=False)
#results.head()
    


KeyboardInterrupt: 

In [5]:
# initialisiere input ('r in open() is for read')

#df_load_bus = pd.DataFrame()
#df_load_variation = pd.DataFrame()
#df_load_deviation = pd.DataFrame()

# Berechne normales Profil load_profil

load_bus = pd.read_excel('load_bus_spring.xlsx')
#load_bus = pd.read_excel('load_bus_summer.xlsx')
#load_bus = pd.read_excel('load_bus_fall.xlsx')
#load_bus = pd.read_excel('load_bus_winter.xlsx')

load_profil = pd.DataFrame(columns=['b'] + ['t'+str(i) for i in range(1,97)])
load_profil['b'] = load_bus['b']
for i in range(1,97):
    load_profil['t'+str(i)] = df_profil['profil'][i-1] * load_bus['Active demand']

                                       
load_profil.head()
#load_profil.to_excel("check.xlsx", index=False)

,b,t1,t2,t3,t4,t5,t6,t7,t8,t9,...,t87,t88,t89,t90,t91,t92,t93,t94,t95,t96
0,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2,0.056829,0.056422,0.055567,0.055574,0.055407,0.055394,0.055480,0.055378,0.055003,...,0.064111,0.063237,0.061540,0.060512,0.059784,0.058837,0.058748,0.058111,0.058035,0.057488
2,3,0.051038,0.050672,0.049904,0.049910,0.049761,0.049749,0.049826,0.049734,0.049397,...,0.057578,0.056793,0.055269,0.054345,0.053691,0.052841,0.052761,0.052189,0.052121,0.051629
3,4,0.067894,0.067407,0.066386,0.066394,0.066195,0.066179,0.066282,0.066160,0.065712,...,0.076594,0.075550,0.073522,0.072294,0.071424,0.070292,0.070186,0.069425,0.069335,0.068681
4,5,0.034037,0.033793,0.033281,0.033285,0.033186,0.033178,0.033229,0.033168,0.032943,...,0.038399,0.037875,0.036859,0.036243,0.035807,0.035240,0.035186,0.034805,0.034760,0.034432


In [6]:
# Berechne die Lastabweichungen zum normalen Profil df_profil
load_variation = pd.DataFrame()

# Iteration über alle Szenarien
for scenario in results.columns:
    results[scenario] = results[scenario].fillna(0)
    load_variation_scenario = pd.DataFrame(columns=['Scenario','b'] + ['t'+ str(i) for i in range(1,97)])
    load_variation_scenario['Scenario'] = [scenario]*33
    load_variation_scenario['b'] = load_bus['b']
    for i in range(1,97):
        load_variation_scenario['t'+ str(i)] = results[scenario][i-1] * load_bus['Active demand'] - load_profil['t'+str(i)]
#        load_variation_scenario.subtract(load_profil)
    load_variation = pd.concat([load_variation,load_variation_scenario], ignore_index=True )
    
# umbennenung für Kompatibilität mit Gams   
load_variation.rename(columns={'t'+str(i): str(i) for i in range(1,97)}, inplace=True)
    
#load_variation.head()
load_variation.to_excel("Data.xlsx", index=False)